# Exercise 03 - Tapping into PyArrow 🏹

By now you should have noticed something. Did you notice it?

_You haven't had to touch anything Apache Arrow specific!_

This is the ideal way we embrace Apache Arrow in our platform. The end-user shouldn't know or care of its existance, just like with Neo4j Bolt/PackStream protocols. It should fade into the background and get out of their way.

## So why Dig Deeper? 🤔

There are two reasons (other than education) for _not_ using the GDS Python Client:

1. You want to scale out you data load across multiple disparate workers. (Think Apache Spark or Apache Beam.)

2. You want to construct a _Database_, not a GDS Projection/Graph. (We'll cover this in _Exercise 04_.)


### The Import Protocol

Whether your building a graph projection or a database, the protocol we follow is the same.

![import-protocol.png](./img/import-protocol.png)

Each of these steps is achieved by a combination of Apache Arrow Flight's RPC calls and streaming API.


### The Who and the What Now? 😖

You shouldn't have to worry about those details! The protocol, however, is _unique to Neo4j_ and _**is** something you should learn_!

So let's dive in!

---
<br><br>

## neo4j_arrow

We're now going to switch to using a Field project authored by `dave.voutila@` that exposes the _Apache PyArrow SDK_ in a way to simplify leveraging the GDS Apache Arrow Flight Service we've been using behind the scenes this entire time.

Currently, the project is in the `neo4j-field` Github repo and publically accessible. This means you can easily install it via `pip install` and pointing at the releast tarball/zipfile you want to use.

> Q: _"Is this supported?"_

> A: **Nope!** This is a field project and provided as-is. It's provided as a reference implementation.

> Q: _"Will this be in PyPI?"_

> A: **Nope!** Dave has zero interest in that.

In [ ]:
%%capture
%pip install https://github.com/neo4j-field/neo4j_arrow/releases/download/0.2.0/neo4j_arrow-0.2.0.tar.gz
%pip install https://github.com/neo4j-field/checker/releases/download/0.4.1/checker-0.4.1.tar.gz

In [ ]:
import pandas as pd

import pyarrow as pa
import pyarrow.parquet

import answers.checker as c

from neo4j_arrow import Neo4jArrowClient

In [ ]:
# Update this if you're running locally with the provided Docker instances.
USE_TLS = True
NEO4J_HOST = "nodes.neo4j.academy"
NEO4J_URI = f"neo4j{'+s' * int(USE_TLS)}://{NEO4J_HOST}:7687"
NEO4J_AUTH = ("user255", "xxxx")

In [ ]:
client = Neo4jArrowClient(NEO4J_HOST,          # host or ip
                          "Exercise-03",       # graph projection name
                          port=8491,           # Arrow service port
                          tls=USE_TLS,
                          database=NEO4J_AUTH[0], # you might need to change this!
                          user=NEO4J_AUTH[0],
                          password=NEO4J_AUTH[1],
                          concurrency=4        # server-side concurrency (num threads)
)

### Reading Parquet with PyArrow

PyArrow provides a lot of helpful utilities for working with a variety of columnar data formats, like Apache Parquet. 

**IF the data is local**, we don't need to use Pandas! 🐼 Check out https://arrow.apache.org/docs/python/generated/pyarrow.parquet.read_table.html.

Otherwise, we can easily us Pandas to fetch the remote Parquet file and flip it into a PyArrow Table.

In [ ]:
df = pd.read_parquet("https://storage.googleapis.com/neo4j-se-public/training/user.parquet")
users = pa.Table.from_pandas(df)

PyArrow Tables are like Pandas DataFrames, but a bit lower level. They're able to be converted to/from DataFrames, but unless you're going to manipulate the data it's not important to create a DataFrame representation.

---
<br><br>

### Task 1. Read our Relationships into a Table
Now that you've seen the basics of reading a Parquet file into a PyArrow `Table`, repeat the process to load the `referred.parquet` file we used previously in _Exercise 01_.

> Call it `referred`.

In [ ]:
df = pd.read_parquet("https://storage.googleapis.com/neo4j-se-public/training/referred.parquet")
referred = pa.Table.from_pandas(df)
referred

In [ ]:
# Don't change this cell.
c.check_result("Ex 03", "Task 1", referred=referred)

---
<br><br>

## Starting the Projection

We first need to signal to the Neo4j GDS server that we're going to project a new graph. The `Neo4jArrowClient` encapsulates some of our configuration already when we constructed it (e.g. the projection name), so starting the process is a simple method call.

If all is well, the server will respond with a message containing the projection name. (In this case, `Exercise-03`.)

In [ ]:
# First step in the Import Protocol

client.start()

You should receive a response like:

```python
{'name': 'Exercise-03'}
```

We've started the process, but now we need to send data!

The protocol follows our common best practice for building a graph at scale:

1. Load nodes
2. Load relationships

The `Neo4jArrowClient` makes it easy to send one or many PyArrow `Table` instances in parallel: we simply pass the `Table` to `write_nodes()`.

On success, it returns a `tuple` representing: `(number of items, size of data sent in bytes)`

In [ ]:
client.write_nodes(users)

---
<br><br>

### 🛎️ Nodes are Done!

Recall that the purpose of using something lower-level than the GDS Python Client is we may have multiple, disparate workers sending nodes to the server.

How does the server find out we're done sending it nodes?

Simple: we tell it we're done!

> In practice, if using a platform like Apache Beam or Apache Spark, you'll want the orchestration node to send this signal so it's sent _once and only once_.

Upon success, we receive a similar message to `start()` but now we get information on the number of nodes received by the server.

In [ ]:
client.nodes_done()

---
<br><br>

### Task 2. Send our Edges to GDS

Now it's your turn! Send the edges to the server. Use either the `neo4j_arrow` code or just the code completion in your notebook to find the method to call.

> Hint: it's very similar to how we write nodes.

Store the response in an object called `result`.

In [ ]:
result = client.write_edges(referred)
result

In [ ]:
# Don't change this cell.
c.check_result("Ex 03", "Task 2", result=result)

---
<br><br>

### Task 3. Signal we're Done Sending Edges

To complete the import protocol, just like we did with nodes, we need to tell the server we're done sending edges.

On success, we'll receive back a similar message as when we signaled nodes were done, but instead of a node count we get the observed relationship count.

Your turn! Tell the server we're done and store the reponse in `result` again.

In [ ]:
result = client.edges_done()
result

In [ ]:
# Don't change this cell.
c.check_result("Ex 03", "Task 3", result=result)

---
<br><br>

### Validate our Work

Since the projection is now live in GDS, you can access it with the GDS Python Client like you would any other projection.

In this case, we expect to see a projection like:

```python
{
    'nodes': (['User'], 33732),
    'edges': (['REFERRED'], 1870)
}
```

In [ ]:
from graphdatascience import GraphDataScience

gds = GraphDataScience(NEO4J_URI, auth=NEO4J_AUTH)
gds.set_database(NEO4J_AUTH[0])

G = gds.graph.get("Exercise-03")

{ "nodes": (G.node_labels(), G.node_count()), "edges": (G.relationship_types(), G.relationship_count()) }

---
<br><br>

In [ ]:
G.drop()